In [8]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
from rnn_classes import Dog, DogInput, Race, Races, GRUNet
from raceDB import build_dataset, build_pred_dataset
import importlib

def build_model_from_save(hidden_size, model_save_file):

    model_dict = torch.load(model_save_file)
    model_size = model_dict['model_state_dict']['gru1.weight_ih'].shape[1]
    model = GRUNet(model_size, hidden_size)
    model.load_state_dict(model_dict['model_state_dict'])

    return model

def flatten(l):
    return [item for sublist in l for item in sublist]

def generate_predictions(model, predDB):

    batch_size = 10
    len_test = len(predDB.raceIDs)-batch_size
    last = 0



    output_dict = {}
    output_dict['race_id'] = []
    output_dict['track'] = []
    output_dict['race_date'] = []
    output_dict['race_time'] = []
    output_dict['dog_name'] = []
    # output_dict['dog_box'] = []
    # output_dict['probabilty'] = []

    with torch.no_grad():
        for i in trange(0,len_test,batch_size, leave=False):
            races_idx = range(last,last+batch_size)
            last = i
            race = predDB.get_race_input(races_idx)
            print(race)
            print([r.raceid for r in race])
            X = race
            # y = torch.stack([x.classes for x in race])
            output = model(X)

            output_dict['race_id'].extend(flatten([[x.raceid]*8 for x in race]))
            output_dict['track'].extend(flatten([[x.track_name]*8 for x in race]))
            output_dict['race_date'].extend(flatten([[x.race_date]*8 for x in race]))
            output_dict['race_time'].extend(flatten([[x.race_time]*8 for x in race]))
            output_dict['dog_name'].extend([x.dog.dog_name for x in race for x in x.dogs])
            # output_dict['dog_box'].extend([x. for x in race])
            # output_dict['probabilty'].extend([x. for x in race])
            break

    for x in output_dict.values():
        print(len(x))
    df_pred = pd.DataFrame(output_dict)
    
    return df_pred



In [9]:
model_file = r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\savedmodel\test NZ GRU saver\test NZ GRU saver_450.pt"
hidden_size = 64

model = build_model_from_save(64, model_file)

# pred_db_file = open( r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\prediction validation 2023-01.npy', 'rb')
# predDB = build_dataset(pred_db_file, hidden_size)

pred_db_file =  open(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\Model Prediction\pred_df 2023-01-12.npy', 'rb')
predDB = build_pred_dataset(pred_db_file, hidden_size)
predDB.to_cpu()
df_pred = generate_predictions(model, predDB)


  0%|          | 0/1248 [00:00<?, ?it/s]

  0%|          | 0/165 [00:00<?, ?it/s]

number of races = 165, number of unique dogs = 1248


  0%|          | 0/16 [00:00<?, ?it/s]

[<rnn_classes.Race object at 0x00000264827BE940>, <rnn_classes.Race object at 0x00000264827BE8E0>, <rnn_classes.Race object at 0x00000264827BEC70>, <rnn_classes.Race object at 0x00000264FFADF1F0>, <rnn_classes.Race object at 0x00000264827BEAF0>, <rnn_classes.Race object at 0x00000264FFADF6A0>, <rnn_classes.Race object at 0x00000264827BEAC0>, <rnn_classes.Race object at 0x00000264FFADF520>, <rnn_classes.Race object at 0x00000264827BED60>, <rnn_classes.Race object at 0x00000264FFADF2B0>]
['860650718', '860650729', '860650730', '860650731', '860650732', '860650733', '860650734', '860650735', '860650736', '860650737']
80
80
80
80
80


In [36]:
races_idx = range(0,165)
race = predDB.get_race_input(races_idx)
s = race[0]

In [37]:
[d.dog.dog_name for d in s.dogs]

['LAUNCH CODE',
 'ASTON MONITOR',
 'PROMISES BROKEN',
 'NICOLA ROSE',
 'KIMBA NEO',
 'CONDOR AVENUE',
 'REDEEMER RUSLIE',
 'ZIPPING LUDA']

In [38]:
[d.prevrace_id for d in s.dogs]

[-1, -1, -1, '856426443', '859247068', -1, -1, -1]

In [27]:
dog_stats_file = open( r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\dog_stats_df_FASTTRACK new margins.npy", 'rb')
full_df = pd.DataFrame(pickle.load(dog_stats_file))

In [29]:
full_df[full_df['raceid']=='857094793']

,dogid,dog_name,raceid,race_grade,date,trackOHE,track_name,dist,stats,place,startprice,margin,bfSP,runtime


In [30]:
full_df

,dogid,dog_name,raceid,race_grade,date,trackOHE,track_name,dist,stats,place,startprice,margin,bfSP,runtime
0,314368991,CALLUM ARI,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[5, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",1.0,11.5,0.00,19.500000,19.39
1,314367487,RIO HUSTLER,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[2, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",2.0,6.3,0.89,7.200000,19.45
2,291188717,THURSDAY THUNDER,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[8, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",3.0,38.6,1.93,40.000000,19.52
3,296096016,ANNIHILATION,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",4.0,7.9,2.01,10.456086,19.53
4,282125654,GRUMPY DEREK,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[7, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",5.0,5.4,3.33,7.543589,19.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020364,497943904,ARBY'S LUCY,849837292,Mixed 4/5,2022-12-01,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Ballarat,545.0,"[5, 0.05867687856061126, 0.0007530556145847705...",6.0,44.7,10.70,NaN,31.70
1020365,514250455,ZIPPY AUDI,849837292,Mixed 4/5,2022-12-01,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Ballarat,545.0,"[1, 0.05803356290858806, 0.0006592486003574619...",4.0,19.0,8.97,39.410868,31.58
1020366,472135386,HILL TOP JACK,849837292,Mixed 4/5,2022-12-01,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Ballarat,545.0,"[2, 0.05758222906907673, 0.0007933659014028218...",1.0,1.2,0.00,1.390000,30.95
1020367,530139164,FIGHTING BOXER,849837292,Mixed 4/5,2022-12-01,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Ballarat,545.0,"[4, 0.05792380953464083, 0.000939278472760607,...",5.0,16.2,9.73,19.000000,31.63


In [32]:
prev_results_file = r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\results-df-merged-prices.npy"

form = pd.DataFrame(pickle.load(open(prev_results_file, "rb")))

In [39]:
form

,@id_x,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,...,margin,weigth,dateF,dog_name,EVENT_ID,EVENT_DT,SELECTION_ID,BSP,dog,MENU_HINT
22541,571660541,5.0,KRUGER LASS,4,4,28.7,48.2,None,12.26,5.57,...,12.26,28.7,2023-01-07,KRUGER LASS,NaN,NaN,NaN,NaN,NaN,NaN
15575,650078197,5.0,YARRAMUNDI PAUL,5,5,31.3,8.0,None,5.50,3.14,...,5.50,31.3,2023-01-07,YARRAMUNDI PAUL,NaN,NaN,NaN,NaN,NaN,NaN
15573,647616151,3.0,LEN ME DAD,3,3,34.2,5.0,None,1.75,0.43,...,1.75,34.2,2023-01-07,LEN ME DAD,NaN,NaN,NaN,NaN,NaN,NaN
15572,647616171,2.0,WINLOCK PIPER,1,1,32.9,4.2,None,1.25,1.29,...,1.25,32.9,2023-01-07,WINLOCK PIPER,NaN,NaN,NaN,NaN,NaN,NaN
15571,714917075,1.0,ROCKSTAR APOLLO,2,2,35.2,2.2,None,1.25,None,...,0.00,35.2,2023-01-07,ROCKSTAR APOLLO,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4869,599869371,7.0,FEELING FANCY,4,9,24.0,8.1,None,5.20,0.45,...,5.20,24.0,2022-12-15,FEELING FANCY,207635588,15-12-2022 04:41,51032906,14.210378,Feeling Fancy,NZL / Addington (NZL) 15th Dec
4870,523685280,8.0,BARBARA'S ANGEL,7,7,34.2,8.4,None,5.50,0.27,...,5.50,34.2,2022-12-15,BARBARA'S ANGEL,NaN,NaN,NaN,NaN,NaN,NaN
4871,515900280,1.0,OPAWA PIP,6,6,27.3,22.4,None,1.30,None,...,0.00,27.3,2022-12-15,OPAWA PIP,207635596,15-12-2022 06:28,40014823,29.000000,Opawa Pip,NZL / Addington (NZL) 15th Dec
4872,550747888,2.0,GOLDSTAR PORTIA,1,1,26.4,1.7,None,1.30,1.17,...,1.30,26.4,2022-12-15,GOLDSTAR PORTIA,207635596,15-12-2022 06:28,41320989,1.950000,Goldstar Portia,NZL / Addington (NZL) 15th Dec


In [34]:
form[form['RaceId']=='856426443']

,@id_x,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,...,margin,weigth,dateF,dog_name,EVENT_ID,EVENT_DT,SELECTION_ID,BSP,dog,MENU_HINT
9439,611903871,4.0,JENNA,6,6,30.0,17.8,None,2.61,2.40,...,2.61,30.0,2022-12-18,JENNA,207754667,18-12-2022 06:33,51010044,30.073158,Jenna,AUS / Sale (AUS) 18th Dec
9440,580792782,5.0,FEEL LIKE FLYING,4,4,26.4,9.5,None,4.91,2.30,...,4.91,26.4,2022-12-18,FEEL LIKE FLYING,207754667,18-12-2022 06:33,43970307,10.000000,Feel Like Flying,AUS / Sale (AUS) 18th Dec
9438,665488323,3.0,NICOLA ROSE,7,7,28.4,2.9,None,0.21,0.14,...,0.21,28.4,2022-12-18,NICOLA ROSE,207754667,18-12-2022 06:33,51090901,3.426816,Nicola Rose,AUS / Sale (AUS) 18th Dec
9437,679825446,2.0,RIO RIPLEY,1,1,37.8,16.3,None,0.07,0.07,...,0.07,37.8,2022-12-18,RIO RIPLEY,207754667,18-12-2022 06:33,51090899,13.268090,Rio Ripley,AUS / Sale (AUS) 18th Dec
9436,631806094,1.0,PATH OF GLORY,8,8,32.8,9.1,None,0.07,None,...,0.00,32.8,2022-12-18,PATH OF GLORY,207754667,18-12-2022 06:33,51090902,10.000000,Path Of Glory,AUS / Sale (AUS) 18th Dec
9441,570998225,6.0,DR. LINDA MAY,2,2,26.2,8.7,None,5.69,0.77,...,5.69,26.2,2022-12-18,DR. LINDA MAY,NaN,NaN,NaN,NaN,NaN,NaN
9442,635302591,7.0,ON THE CUSP,5,9,27.2,41.0,None,8.59,2.90,...,8.59,27.2,2022-12-18,ON THE CUSP,207754667,18-12-2022 06:33,50354848,172.343413,On The Cusp,AUS / Sale (AUS) 18th Dec
